Alohan'ny mamerina dia avereno atao Run ny notebook iray manontolo. Ny fanaovana azy dia redémarrena mihitsy ny kernel aloha (jereo menubar, safidio **Kernel$\rightarrow$Restart Kernel and Run All Cells**).

Izay misy hoe `YOUR CODE HERE` na "YOUR ANSWER HERE" ihany no fenoina. Afaka manampy cells vaovao raha ilaina. Aza adino ny mameno references eo ambany raha ilaina.

## References
Eto ilay references rehetra no apetraka

https://stackoverflow.com/questions/41663874/cs231n-how-to-calculate-gradient-for-softmax-loss-function?fbclid=IwAR2AoT45U6AWM7PCZHPSL1g83o_yoQcwlZPZKd3SjGULm8AAUf6aHtk4NdQ

https://github.com/egeyelken/ComputerVisionDL/blob/efc0ee545591c366b815778eb5e0782a60a8577d/eyelken17_61742_assignment1/comp451/classifiers/softmax.py

https://github.com/egeyelken/ComputerVisionDL/blob/efc0ee545591c366b815778eb5e0782a60a8577d/eyelken17_61742_assignment1/comp451/classifiers/k_nearest_neighbor.py

In [1]:
# AZA MANAMPY CODE ATO FA MNAOVA CELLULE VAOVAO

from random import randrange
import numpy as np
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.linear_model import HuberRegressor
from sklearn.datasets import load_boston, load_diabetes, load_iris, load_digits
from scipy.special import huber
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

def grad_check_sparse(f, x, analytic_grad, num_checks=12, h=1e-5, error=1e-9):
    """
    sample a few random elements and only return numerical
    in this dimensions
    """

    for i in range(num_checks):
        ix = tuple([randrange(m) for m in x.shape])

        oldval = x[ix]
        x[ix] = oldval + h  # increment by h
        fxph = f(x)  # evaluate f(x + h)
        x[ix] = oldval - h  # increment by h
        fxmh = f(x)  # evaluate f(x - h)
        x[ix] = oldval  # reset

        grad_numerical = (fxph - fxmh) / (2 * h)
        grad_analytic = analytic_grad[ix]
        rel_error = abs(grad_numerical - grad_analytic) / (
            abs(grad_numerical) + abs(grad_analytic)
        )
        print(
            "numerical: %f analytic: %f, relative error: %e"
            % (grad_numerical, grad_analytic, rel_error)
        )
        assert rel_error < error

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# Robust linear regression - Huber loss

In [2]:
data = load_boston()
X_train1, y_train1 = data.data, data.target
w1 = np.random.randn(X_train1.shape[1]) * 0.0001
b1 = np.random.randn(1) * 0.0001

/home/erwan/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing da

In [3]:
def huber_loss_naive(w, b, X, y, epsilon=1.35, alpha=0.0001):
    """
    Huber loss for all observations
    
    Inputs:
    - w: array of shape (D,) containing weights
    - b: float bias 
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels 
    - epsilon: float
    - alpha: regularization
    """
    loss = 0.0
    dw = np.zeros_like(w)
    db = 0.0
    
    # YOUR CODE HERE
    n = X.shape[0]
    
    for i in range (0, n) :
        r = (w.T@X[i] +b - y[i])
        
        if abs(r)<=epsilon :
            loss += ((r**2)/2)/n
            dw += (X[i] * r)/n
            db += r/n
        else :
            loss += (epsilon * abs(r) - (epsilon**2)/2)/n
            dw += (epsilon * np.sign(r) * X[i])/n
            db += (epsilon * np.sign(r))/n
            
    loss += alpha * (np.linalg.norm(w, 2))**2
    dw += alpha * 2.0 * w
                     
    return loss, dw, np.array(db).reshape(1,)

## without regularization

In [4]:
loss, dw1, db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

Gradient check w
numerical: -24.914970 analytic: -24.914970, relative error: 8.828679e-12
numerical: -15.340909 analytic: -15.340909, relative error: 2.129788e-11
numerical: -15.340909 analytic: -15.340909, relative error: 2.129788e-11
numerical: -551.120158 analytic: -551.120158, relative error: 3.187076e-13
numerical: -15.340909 analytic: -15.340909, relative error: 2.129788e-11
numerical: -8.484256 analytic: -8.484256, relative error: 1.047315e-11
numerical: -12.891700 analytic: -12.891700, relative error: 2.147269e-11
numerical: -481.509943 analytic: -481.509943, relative error: 7.998642e-13
numerical: -24.914970 analytic: -24.914970, relative error: 8.828679e-12
numerical: -481.509943 analytic: -481.509943, relative error: 7.998642e-13
numerical: -551.120158 analytic: -551.120158, relative error: 3.187076e-13
numerical: -4.878257 analytic: -4.878257, relative error: 3.509817e-10
numerical: -24.914970 analytic: -24.914970, relative error: 8.828679e-12
numerical: -4.878257 analytic:

 ## with regularization

In [6]:
loss, dw1, db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

Gradient check w
numerical: -0.748785 analytic: -0.748785, relative error: 8.990586e-10
numerical: -92.576395 analytic: -92.576395, relative error: 1.763455e-12
numerical: -15.034591 analytic: -15.034591, relative error: 1.248753e-11
numerical: -24.914745 analytic: -24.914745, relative error: 9.714344e-12
numerical: -4.878287 analytic: -4.878287, relative error: 3.562240e-10
numerical: -24.914745 analytic: -24.914745, relative error: 9.714344e-12
numerical: -12.891214 analytic: -12.891214, relative error: 2.348401e-11
numerical: -4.878287 analytic: -4.878287, relative error: 3.562240e-10
numerical: -15.341111 analytic: -15.341111, relative error: 1.846606e-11
numerical: -12.891214 analytic: -12.891214, relative error: 2.348401e-11
numerical: -0.748785 analytic: -0.748785, relative error: 8.990586e-10
numerical: -17.081830 analytic: -17.081830, relative error: 5.198053e-11
numerical: -5.122925 analytic: -5.122925, relative error: 2.357137e-10
numerical: -5.122925 analytic: -5.122925, re

In [7]:
def huber_loss_vectorized(w, b, X, y, epsilon=1.35, alpha=0.0001):
    """
    Huber loss for all observations
    
    Inputs:
    - w: array of shape (D,) containing weights
    - b: float bias 
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels 
    - epsilon: float
    - alpha: regularization
    """
    loss = 0.0
    dw = np.zeros_like(w)
    db = 0
    
    n = X.shape[0]
    r = (X@w +b - y)
        
    if abs(np.linalg.norm(r, 1))<=epsilon :
        loss = ((r**2)/2)/n
        dw = (X @ r)/n
        db = r/n
    else :
        loss = (epsilon * abs(np.linalg.norm(r, 1)) - (epsilon**2)/2)/n
        dw = (epsilon * np.sign(r) @ X)/n
        db = (epsilon * np.sign(r))/n
            
    loss += alpha * (np.linalg.norm(w, 2))**2
    dw += alpha * 2.0 * w   
    db = sum(db)
    return loss, dw, np.array(db).reshape(1,)

## without regularization

In [8]:
loss, dw1, db1 = huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

Gradient check w
numerical: -0.093379 analytic: -0.093379, relative error: 6.023259e-10
numerical: -15.340909 analytic: -15.340909, relative error: 7.650619e-12
numerical: -17.081635 analytic: -17.081635, relative error: 4.899177e-12
numerical: -12.891700 analytic: -12.891700, relative error: 8.039401e-13
numerical: -15.340909 analytic: -15.340909, relative error: 7.650619e-12
numerical: -0.093379 analytic: -0.093379, relative error: 6.023259e-10
numerical: -24.914970 analytic: -24.914970, relative error: 1.255864e-11
numerical: -0.093379 analytic: -0.093379, relative error: 6.023259e-10
numerical: -0.748838 analytic: -0.748838, relative error: 9.981577e-11
numerical: -17.081635 analytic: -17.081635, relative error: 4.899177e-12
numerical: -0.748838 analytic: -0.748838, relative error: 9.981577e-11
numerical: -551.120158 analytic: -551.120158, relative error: 3.213892e-13
numerical: -551.120158 analytic: -551.120158, relative error: 3.213892e-13
numerical: -17.081635 analytic: -17.0816

AssertionError: 

## with regularization

In [9]:
loss, dw1, db1 = huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

Gradient check w
numerical: -0.748785 analytic: -0.748785, relative error: 4.986978e-11
numerical: -24.914745 analytic: -24.914745, relative error: 1.167317e-11
numerical: -8.484035 analytic: -8.484035, relative error: 1.945756e-11
numerical: -17.081830 analytic: -17.081830, relative error: 5.214421e-12
numerical: -0.093516 analytic: -0.093516, relative error: 2.241475e-10
numerical: -92.576395 analytic: -92.576395, relative error: 8.028264e-13
numerical: -12.891214 analytic: -12.891214, relative error: 2.814482e-12
numerical: -15.034591 analytic: -15.034591, relative error: 5.234813e-12
numerical: -0.093516 analytic: -0.093516, relative error: 2.241475e-10
numerical: -551.120177 analytic: -551.120177, relative error: 3.295374e-13
numerical: -15.341111 analytic: -15.341111, relative error: 1.048206e-11
numerical: -8.484035 analytic: -8.484035, relative error: 1.945756e-11
numerical: -92.576395 analytic: -92.576395, relative error: 8.028264e-13
numerical: -15.341111 analytic: -15.341111

AssertionError: 

In [10]:
class LinearModel():
    def __init__(self):
        self.w = None
        self.b = None

    def train(self, X, y, learning_rate=1e-3, alpha=0.0001, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape
        
        if self.w is None: # Initialization
            self.w = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            ## echantillonage
            i = np.random.choice (X.shape[0], batch_size)
            ## building matrix X and vector y
            X_batch = X[i, :]
            y_batch = y[i]
            
            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch, alpha)
            loss_history.append(loss)

            # perform parameter update                                                                
            # Update the weights w and bias b using the gradient and the learning rate.          
            self.w = self.w - learning_rate * dw
            self.b = self.b - learning_rate * db
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))
                
        return loss_history

    def predict(self, X):
        pass

    def loss(self, X_batch, y_batch, reg):
        pass

class HuberRegression(LinearModel):
    """ Linear regression """

    def loss(self, X_batch, y_batch, alpha):
        return huber_loss_vectorized(self.w, self.b, X_batch, y_batch, alpha=alpha)
    
    def predict(self, X):
        return X@self.w + self.b

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train1)

sk_model = HuberRegressor(fit_intercept=True)
sk_model.fit(X_train1, y_train1)
sk_pred = sk_model.predict(X_train1)
sk_mse = mean_squared_error(sk_pred, y_train1)

model = HuberRegression()
model.train(X_train1, y_train1, num_iters=75000, batch_size=64, learning_rate=1e-2, verbose=True)
pred = model.predict(X_train1)
mse = mean_squared_error(pred, y_train1)

print("MSE scikit-learn:", sk_mse)
print("MSE gradient descent model :", mse)
assert mse - sk_mse < 1

iteration 0 / 75000: loss 32.174303
iteration 10000 / 75000: loss 3.892807
iteration 20000 / 75000: loss 5.228802
iteration 30000 / 75000: loss 4.275602
iteration 40000 / 75000: loss 5.417389
iteration 50000 / 75000: loss 3.946746
iteration 60000 / 75000: loss 3.263848
iteration 70000 / 75000: loss 3.260043
MSE scikit-learn: 24.04102301055777
MSE gradient descent model : 24.681378410394075


# Multinomial logistic regression

In [12]:
data = load_iris()
X_train2, y_train2 = data.data, data.target

W = np.random.randn(X_train2.shape[1], 3) * 0.0001

In [13]:
def softmax_loss_naive(W, X, y, alpha):
    """
    Softmax loss function WITH FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization 

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W;  same shape as W
    """
    
    # Initialization
    loss = 0.0
    dW = np.zeros_like(W)

    
    # Tandremo ny numeric instability

    # Get shapes
    classes = W.shape[1]
    n = X.shape[0]

    Xt = np.transpose(X)
    Wt = np.transpose(W)
    
    dW = dW.T
    
    for i in range(0, n):
        # Compute vector of scores
        f_i = Wt.dot(Xt[:, i]) # in R^{num_classes}
        #print(f_i)

        # Normalization trick to avoid numerical instability, per http://cs231n.github.io/linear-classify/#softmax
        log_c = np.max(f_i)
        f_i -= log_c

        # Compute loss (and add to it, divided later)
        # L_i = - f(x_i)_{y_i} + log \sum_j e^{f(x_i)_j}
        sum_i = 0.0
        for f_i_j in f_i:
            sum_i += np.exp(f_i_j)
        loss += -f_i[y[i]] + np.log(sum_i)

        # Compute gradient
        # dw_j = 1/num_train * \sum_i[x_i * (p(y_i = j)-Ind{y_i = j} )]
        # Here we are computing the contribution to the inner sum for a given i.
        
        for j in range(classes):
            p = np.exp(f_i[j])/sum_i
            dW[j, :] += (p - (j == y[i])) * Xt[:, i]

    # Compute average
    loss /= n
    dW /= n

  # Regularization
    loss += 0.5 * alpha * np.sum(Wt * Wt)
    dW += alpha*Wt
    
    dW = dW.T
    
    """
    for k in range(0, n):
        scores_diff = np.dot(X, W)[k] - np.max(np.dot(X, W)[k])
        softmax = np.exp(scores_diff) / np.sum(np.exp(scores_diff))
        loss = loss - np.log(softmax[y[k]])
        dW[:,y[k]] -= X[k]
        for i in range(W.shape[1]):
            dW[:, i] += softmax[i] * X[k]
        loss /= X.shape[0]
        dW /= X.shape[0]
    """        
    return loss, dW

## Without regularization

In [14]:
loss, dW = softmax_loss_naive(W, X_train2, y_train2, 0.0)

f = lambda W: softmax_loss_naive(W, X_train2, y_train2, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: -0.166913 analytic: -0.166913, relative error: 2.213164e-10
numerical: 0.027797 analytic: 0.027797, relative error: 2.903960e-10
numerical: -0.166913 analytic: -0.166913, relative error: 2.213164e-10
numerical: 0.278622 analytic: 0.278622, relative error: 3.335686e-10
numerical: 0.317666 analytic: 0.317666, relative error: 3.153628e-11
numerical: 0.027797 analytic: 0.027797, relative error: 2.903960e-10
numerical: 0.278622 analytic: 0.278622, relative error: 3.335686e-10
numerical: -0.275592 analytic: -0.275592, relative error: 8.172165e-11
numerical: -0.123797 analytic: -0.123797, relative error: 3.115548e-10
numerical: -0.275592 analytic: -0.275592, relative error: 8.172165e-11
numerical: -0.030384 analytic: -0.030384, relative error: 4.221851e-09
numerical: -0.166913 analytic: -0.166913, relative error: 2.213164e-10


## With regularization

In [15]:
loss, dW = softmax_loss_naive(W, X_train2, y_train2, 2)

f = lambda W: softmax_loss_naive(W, X_train2, y_train2, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: -0.123468 analytic: -0.123468, relative error: 2.880854e-10
numerical: 0.765254 analytic: 0.765254, relative error: 7.183249e-11
numerical: 0.317544 analytic: 0.317544, relative error: 2.566049e-11
numerical: -0.275895 analytic: -0.275895, relative error: 8.179865e-11
numerical: -0.598010 analytic: -0.598010, relative error: 8.200295e-11
numerical: 0.095980 analytic: 0.095980, relative error: 3.987252e-10
numerical: -0.275895 analytic: -0.275895, relative error: 8.179865e-11
numerical: -0.030006 analytic: -0.030006, relative error: 4.299489e-09
numerical: -0.275895 analytic: -0.275895, relative error: 8.179865e-11
numerical: 0.765254 analytic: 0.765254, relative error: 7.183249e-11
numerical: -0.275895 analytic: -0.275895, relative error: 8.179865e-11
numerical: 0.317544 analytic: 0.317544, relative error: 2.566049e-11


In [16]:
def softmax_loss_vectorized(W, X, y, alpha, fit_intercept=False):
    """
    Softmax loss function WITHOUT FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization 

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W;  same shape as W
    """
    # Initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros_like(W)

    if fit_intercept :
        ## inserting a column of 1 in the features matrix
        X = np.insert(X, 0, 1, axis=1)
    
    n = X.shape[0]
    
    wx = np.dot(X, W) - np.max(np.dot(X, W), axis=1, keepdims=True)

    softmax = np.exp(wx)/np.sum(np.exp(wx), axis=1,keepdims=True)
    loss = np.sum(-np.log(softmax[np.arange(n), y]))
    softmax[np.arange(n), y] -= 1

    dW = np.dot(np.transpose(X), softmax)
    loss /= n
    dW /= n

    loss += alpha * np.sum(W*W)
    dW += alpha * 2 * W

    return loss, dW

## Without regularization

In [17]:
loss, dW = softmax_loss_vectorized(W, X_train2, y_train2, 0.0)

f = lambda W: softmax_loss_vectorized(W, X_train2, y_train2, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: 0.027797 analytic: 0.027797, relative error: 4.900945e-10
numerical: -0.166913 analytic: -0.166913, relative error: 3.543458e-10
numerical: -0.166913 analytic: -0.166913, relative error: 3.543458e-10
numerical: 0.317666 analytic: 0.317666, relative error: 1.406152e-11
numerical: 0.278622 analytic: 0.278622, relative error: 4.730311e-10
numerical: -0.123797 analytic: -0.123797, relative error: 1.770323e-10
numerical: -0.248239 analytic: -0.248239, relative error: 5.176673e-10
numerical: 0.764994 analytic: 0.764994, relative error: 5.461293e-11
numerical: 0.278622 analytic: 0.278622, relative error: 4.730311e-10
numerical: -0.275592 analytic: -0.275592, relative error: 1.899073e-11
numerical: 0.278622 analytic: 0.278622, relative error: 4.730311e-10
numerical: 0.278622 analytic: 0.278622, relative error: 4.730311e-10


## With regularization

In [18]:
loss, dW = softmax_loss_vectorized(W, X_train2, y_train2, 2)

f = lambda W: softmax_loss_vectorized(W, X_train2, y_train2, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: 0.028105 analytic: 0.028105, relative error: 6.355398e-10
numerical: 0.317422 analytic: 0.317422, relative error: 1.978019e-11
numerical: -0.041365 analytic: -0.041365, relative error: 2.559992e-10
numerical: -0.276197 analytic: -0.276197, relative error: 1.861630e-11
numerical: -0.247341 analytic: -0.247341, relative error: 5.078883e-10
numerical: -0.276197 analytic: -0.276197, relative error: 1.861630e-11
numerical: -0.166446 analytic: -0.166446, relative error: 3.622007e-10
numerical: 0.317422 analytic: 0.317422, relative error: 1.978019e-11
numerical: 0.317422 analytic: 0.317422, relative error: 1.978019e-11
numerical: 0.278848 analytic: 0.278848, relative error: 4.733501e-10
numerical: -0.041365 analytic: -0.041365, relative error: 2.559992e-10
numerical: -0.247341 analytic: -0.247341, relative error: 5.078883e-10


## Gradient descent

In [26]:
class LinearModel():
    def __init__(self, fit_intercept=True):
        self.W = None
        self.fit_intercept = fit_intercept

    def train(self, X, y, learning_rate=1e-3, alpha=0, num_iters=100, batch_size=200, verbose=False):
        if self.fit_intercept:
            ## inserting a column of 1 in the features matrix
            X = np.insert(X, 0, 1, axis=1)
            
        N, d = X.shape
        
        C = (np.max(y) + 1) 
        if self.W is None: # Initialization
            self.W = 0.001 * np.random.randn(d, C)

        # Run stochastic gradient descent to optimize W
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            i = np.random.choice (X.shape[0], batch_size)
            ## building matrix X and vector y
            X_batch = X[i, :]
            y_batch = y[i]
            
            # evaluate loss and gradient
            loss, dW = self.loss(X_batch, y_batch, alpha)
            loss_history.append(loss)

            # perform parameter update                                                                
            # Update the weights w using the gradient and the learning rate.          
            self.W = self.W - learning_rate * dW
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))
                
        return loss_history

    def predict(self, X):
        pass

    def loss(self, X_batch, y_batch, reg):
        pass

class MultinomialLogisticRegressor(LinearModel):
    """ Softmax regression """

    def loss(self, X_batch, y_batch, alpha):
        return softmax_loss_vectorized(self.W, X_batch, y_batch, alpha)
    
    def predict(self, X):
        """ 
        Inputs:
        - X: array of shape (N, D) 

        Returns:
        - y_pred: 1-dimensional array of length N, each element is an integer giving the predicted class 
        """
        if self.fit_intercept:
           ## inserting a column of 1 in the features matrix
           X = np.insert(X, 0, 1, axis=1)
            
        y_pred = np.argmax(np.dot(X, self.W), axis = 1)
        return y_pred

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train2 = scaler.fit_transform(X_train2)

sk_model = LogisticRegression(fit_intercept=False)
sk_model.fit(X_train2, y_train2)
sk_pred = sk_model.predict(X_train2)
sk_accuracy = accuracy_score(y_train2, sk_pred)

model = MultinomialLogisticRegressor(fit_intercept=False)
model.train(X_train2, y_train2, num_iters=75000, batch_size=64, learning_rate=1e-3, verbose=True)
pred = model.predict(X_train2)
model_accuracy = accuracy_score(y_train2, pred)

print("Accuracy scikit-learn:", sk_accuracy)
print("Accuracy gradient descent model :", model_accuracy)
assert sk_accuracy - model_accuracy < 0.01

iteration 0 / 75000: loss 1.098873
iteration 10000 / 75000: loss 0.443696
iteration 20000 / 75000: loss 0.329909
iteration 30000 / 75000: loss 0.240563
iteration 40000 / 75000: loss 0.336661
iteration 50000 / 75000: loss 0.336973
iteration 60000 / 75000: loss 0.434587
iteration 70000 / 75000: loss 0.428090
Accuracy scikit-learn: 0.86
Accuracy gradient descent model : 0.8666666666666667


In [28]:
sk_model = LogisticRegression(fit_intercept=True)
sk_model.fit(X_train2, y_train2)
sk_pred = sk_model.predict(X_train2)
sk_accuracy = accuracy_score(y_train2, sk_pred)

model = MultinomialLogisticRegressor(fit_intercept=True)
model.train(X_train2, y_train2, num_iters=75000, batch_size=64, learning_rate=1e-3, verbose=True)
pred = model.predict(X_train2)
model_accuracy = accuracy_score(y_train2, pred)

print("Accuracy scikit-learn:", sk_accuracy)
print("Accuracy gradient descent model :", model_accuracy)
assert sk_accuracy - model_accuracy < 0.02

iteration 0 / 75000: loss 1.096789
iteration 10000 / 75000: loss 0.282968
iteration 20000 / 75000: loss 0.221421
iteration 30000 / 75000: loss 0.254042
iteration 40000 / 75000: loss 0.152881
iteration 50000 / 75000: loss 0.204131
iteration 60000 / 75000: loss 0.137046
iteration 70000 / 75000: loss 0.132937
Accuracy scikit-learn: 0.9733333333333334
Accuracy gradient descent model : 0.96


# K-Nearest Neighbor

## Computing distances

In [29]:
data = load_digits()
X_train3, y_train3 = data.data, data.target
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_train3, y_train3, test_size=0.33, random_state=2)

def get_distances_two_loops_with_norm(X_train, X_test):
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))
    for i in range(num_test):
        for j in range(num_train):
            distances[i, j] = np.linalg.norm(X_test[i] - X_train[j])
    return distances

In [30]:
def get_distances_two_loops(X_train, X_test):
    """
    Compute the distance between each test point in X_test and each training point
    in X_train 

    Inputs:
    - X_test: array of shape (num_test, D) 

    Returns:
    - distances: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between 
    the ith test point and the jth training point.
    """
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))
    for i in range(num_test):
        for j in range(num_train):
            # Ataovy ao anaty distances[i, j] ny distance entre ith test point sy th training point
            # Aza manao boucle instony ato anatiny
            # TSY MAHAZO MAMPIASA np.linalg.norm() :D
            distances[i][j] = np.sqrt(np.sum((X_test[i]-X_train[j])**2))
                                      
    return distances

In [31]:
distances = get_distances_two_loops(X_train3, X_test3)
true_distances = get_distances_two_loops_with_norm(X_train3, X_test3)

difference = np.linalg.norm(distances - true_distances, ord='fro')

print(difference)
assert difference < 1e-10

0.0


In [32]:
def compute_distances_one_loop(X_train, X_test):
    """
    Compute the distance between each test point in X_test and each training point
    in X_train 

    Inputs:
    - X_test: array of shape (num_test, D) 

    Returns:
    - dists: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between 
    the ith test point and the jth training point.
    """
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))
    for i in range(num_test):
        # Ataovy ao anaty dists[i, j] ny distance entre ith test point sy th training point
        # Aza manao boucle instony ato anatiny
        # TSY MAHAZO MAMPIASA np.linalg.norm() :D
        distances[i] = np.sqrt(np.sum((X_test[i] - X_train)**2, axis=1))
        
    return distances    

In [33]:
distances = compute_distances_one_loop(X_train3, X_test3)
true_distances = get_distances_two_loops_with_norm(X_train3, X_test3)

difference = np.linalg.norm(distances - true_distances, ord='fro')

print(difference)
assert difference < 1e-10

0.0


In [34]:
def get_distances_zero_loop(X_train, X_test):
    """
    Compute the distance between each test point in X_test and each training point
    in X_train 

    Inputs:
    - X_test: array of shape (num_test, D) 

    Returns:
    - distances: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between 
    the ith test point and the jth training point.
    """
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))  
    # Ataovy ao anaty dists[i, j] ny distance entre ith test point sy th training point
    # Aza manao boucle instony
    # TSY MAHAZO MAMPIASA np.linalg.norm() NA FONCTIONS AO AMIN'NY SCIPY :D
            

    distances = np.sqrt(np.sum(X_test**2, axis = 1)[:, np.newaxis] + np.sum(X_train**2, axis = 1) - 2 * (np.dot(X_test, np.transpose(X_train))))
    
    return distances

In [35]:
distances = get_distances_zero_loop(X_train3, X_test3)
true_distances = get_distances_two_loops_with_norm(X_train3, X_test3)

difference = np.linalg.norm(distances - true_distances, ord='fro')

print(difference)
assert difference < 1e-10

0.0


## K-Nearest Neighbor (knn) classifier

In [36]:
class KNearestNeighborClassifier():
    """ kNN classifier using L2 distance """

    def __init__(self, k=1):
        """
        Inputs:
        - k: number of nearest neighbors that vote for the predicted labels.
        """
        self.k = k

    def fit(self, X, y):
        """
        Train the classifier. Just memorize the training data.

        Inputs:
        - X: array of shape (num_train, D) 
        - y: array of shape (N,) 
        """
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        """
        Predict labels for test data using this classifier.

        Inputs:
        - X: array of shape (num_test, D) 

        Returns:
        - y: array of shape (num_test,) 
        """
        distances = get_distances_zero_loop(self.X_train, X)
        return self.predict_labels(distances)

    def predict_labels(self, distances):
        """
        Given a matrix of distances between test points and training points,
        predict a label for each test point.

        Inputs:
        - distances: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between 
        the ith test point and the jth training point.

        Returns:
        - y:  array of shape (num_test,) 
        """
        num_test = distances.shape[0]
        y_pred = np.zeros(num_test)
        for i in range(num_test):
            # list storing the labels of the k nearest neighbors to the ith test point.
            closest_y = []

            # Ampidirina ao anaty closest_y ny labels an'ny k neighbors akaiky indrindra
            # Jereo fampiasana np.argsort
            closest_y = self.y_train[np.argsort(distances[i])][0:self.k]

            # Tadiavo ny label betsaka indrindra dia iny no atao prediction
            # Raha misy mitovy dia izay label kely raisina
            y_pred[i] = np.bincount(closest_y).argmax()

        return y_pred

In [37]:
sk_model = KNeighborsClassifier(n_neighbors=3)
sk_model.fit(X_train3, y_train3)
sk_pred = sk_model.predict(X_test3)
sk_accuracy = accuracy_score(y_test3, sk_pred)

model = KNearestNeighborClassifier(k=3)
model.fit(X_train3, y_train3)
pred = model.predict(X_test3)
model_accuracy = accuracy_score(y_test3, pred)

print("Accuracy scikit-learn:", sk_accuracy)
print("Accuracy gradient descent model :", model_accuracy)
assert sk_accuracy - model_accuracy < 1e-10

/home/erwan/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Accuracy scikit-learn: 0.9831649831649831
Accuracy gradient descent model : 0.9831649831649831


## cross-validation

In [38]:
num_folds = 5
k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]

X_train_folds = []
y_train_folds = []

# Split up the data into folds
# X_train_folds and y_train_folds lits of length num_folds

X_train_folds = np.array_split(X_train3, num_folds)
y_train_folds = np.array_split(y_train3, num_folds)


# A dictionary of length num_folds holding the accuracies for different values of k 
k_to_accuracies = {}

# Ataovy ary ilay k-fold cross validation 
# Atao ao anaty k_to_accuracies ny accuracy isaky ny valeur k

## We will try successivly all the hyper-parameter
for k in k_choices :
    model = KNearestNeighborClassifier(k)
    k_to_accuracies[k] = []
    ## We will now train or/and test
    for i in range(0, num_folds) :
        ## define our test
        Xtest = X_train_folds[i]
        ytest = y_train_folds[i]
        ## define our train (matrix) by removing the test
        Xtrain = np.concatenate(np.delete(X_train_folds, i, axis=0))
        ytrain = np.concatenate(np.delete(y_train_folds, i, axis=0))
        ## train and predict, accuracy
        model.fit(Xtrain, ytrain)
        model_accuracy = accuracy_score(ytest, model.predict(Xtest))
        
        ## adding it to our dictionnary
        k_to_accuracies[k].append(model_accuracy)

/home/erwan/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:4454: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)
/home/erwan/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:4454: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)
/home/erwan/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:4454: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is depreca

/home/erwan/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:4454: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)
/home/erwan/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:4454: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)
/home/erwan/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:4454: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is depreca

In [39]:
for k in sorted(k_to_accuracies):
    for accuracy in k_to_accuracies[k]:
        print('k = %d, accuracy = %f' % (k, accuracy))

k = 1, accuracy = 0.991701
k = 1, accuracy = 0.983402
k = 1, accuracy = 0.983402
k = 1, accuracy = 0.970833
k = 1, accuracy = 0.995833
k = 3, accuracy = 0.987552
k = 3, accuracy = 0.983402
k = 3, accuracy = 0.987552
k = 3, accuracy = 0.970833
k = 3, accuracy = 0.983333
k = 5, accuracy = 0.991701
k = 5, accuracy = 0.983402
k = 5, accuracy = 0.987552
k = 5, accuracy = 0.975000
k = 5, accuracy = 0.987500
k = 8, accuracy = 0.979253
k = 8, accuracy = 0.970954
k = 8, accuracy = 0.987552
k = 8, accuracy = 0.966667
k = 8, accuracy = 0.983333
k = 10, accuracy = 0.983402
k = 10, accuracy = 0.970954
k = 10, accuracy = 0.979253
k = 10, accuracy = 0.966667
k = 10, accuracy = 0.975000
k = 12, accuracy = 0.979253
k = 12, accuracy = 0.975104
k = 12, accuracy = 0.979253
k = 12, accuracy = 0.958333
k = 12, accuracy = 0.979167
k = 15, accuracy = 0.983402
k = 15, accuracy = 0.962656
k = 15, accuracy = 0.983402
k = 15, accuracy = 0.966667
k = 15, accuracy = 0.958333
k = 20, accuracy = 0.970954
k = 20, accu